In [1]:
!pip install google-cloud-aiplatform==1.1.1
!pip install google-cloud-bigquery==2.20.0
!pip install kfp==1.6.3

In [ ]:
# Automatically restart kernel after installs, only for development environnment
#import IPython

#app = IPython.Application.instance()
#app.kernel.do_shutdown(True)

In [3]:
!python3 -c "import kfp; print('KFP SDK version: {}'.format(kfp.__version__))"

KFP SDK version: 1.6.3


In [4]:
PROJECT_ID = 'cloud-sandbox-danielw'
GCS_BUCKET = 'gs://ai-demo-uscentral'
REGION = "us-central1"

SQL_QUERY = """
    SELECT name, SUM(number) as total_people
    FROM `bigquery-public-data.usa_names.usa_1910_2013`
    WHERE state = 'TX'
    GROUP BY name, state
    ORDER BY total_people DESC
    LIMIT 20
"""

In [5]:
import json
from datetime import datetime
from kfp import dsl
from kfp.v2.dsl import (
    component,
    Input,
    Output,
    Dataset,
    Metrics,
)
from kfp.v2 import compiler
from kfp.v2.google.client import AIPlatformClient

In [12]:
from typing import NamedTuple

@component(base_image="python:3.7", packages_to_install=["google-cloud-bigquery==2.20.0"])
def custom_etl_op(
    project_id: str,
    sql_query: str
) -> NamedTuple('Outputs', [
    ('Success', bool),  
    ('Count', int)
]):
    """
        Comments
    """
    import logging
    from google.cloud import bigquery
    
    logging.getLogger().setLevel(logging.INFO)
    
    success = False
    count = 0
    
    logging.info("project_id = {}".format(project_id))
    logging.info("sql_query = {}".format(sql_query))
    
    client = bigquery.Client(project=project_id)
    
    query_job = client.query(sql_query) 
    result = query_job.result() 
    
    count = result.total_rows
    logging.info("total row count = {}".format(count))
    
    success = True
    
    return (success, count)
    
#    Outputs = NamedTuple('Outputs', ['Success', 'Count'])
#    return Outputs(
#        Success = success,
#        Count = count
#    )

In [13]:
@component
def dummy_op(
    success: bool,
    count: int
):
  import logging
  
  logging.info("success = {}".format(success))
  logging.info("count = {}".format(count))

In [14]:
TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

PIPELINE_DEF = "etl_pipeline.json"
PIPELINE_ROOT = "gs://ai-demo-uscentral/pipeline_root/etl_pipeline/{}".format(TIMESTAMP)

In [15]:
@dsl.pipeline(
  name = 'etl-pipeline',
  description = 'A pipeline that performs ETL through BigQuery',
  pipeline_root = PIPELINE_ROOT
)
def etl_pipeline(
  project_id: str,
  sql_query: str
):
  # ETL 
  etl_step = custom_etl_op(project_id, sql_query)

  # dummy 
  dummy_step = dummy_op(
      etl_step.outputs["Success"],
      etl_step.outputs["Count"]
  )

In [16]:
# compile the pipeline
compiler.Compiler().compile(
    pipeline_func = etl_pipeline, 
    package_path = PIPELINE_DEF
)

# connect to Vertex AI platform
api_client = AIPlatformClient(
    project_id = PROJECT_ID,
    region = REGION,
)

# Specify pipeline argument values
pipeline_arguments = {'project_id': PROJECT_ID, 'sql_query': SQL_QUERY}

# submit the pipeline
response = api_client.create_run_from_job_spec(
    job_spec_path = PIPELINE_DEF, 
    pipeline_root = PIPELINE_ROOT,
    parameter_values = pipeline_arguments
)